# **IMPORTS & PATHS** 

In [43]:
import os
import zipfile
import subprocess
import json
import pandas as pd
from langchain_ollama.llms import OllamaLLM
import shutil


# Paths and model settings\ nZIP_PATH = 'I-Soon-data.zip'      # Path to your downloaded zip file
ZIP_PATH = '0.zip'    # Directory to extract contents
EXTRACT_DIR = 'I-Soon-data'        # Directory to extract contents
ORGANIZED_DIR = 'organized_data'  # Directory to group files by extension
MODEL_NAME = 'llama3.2'             # Local Ollama model identifier
OUTPUT_JSON = 'parsed_md.json'    # Aggregated JSON output
OUTPUT_CSV = 'parsed_md.csv'      # CSV output for DataFrame

# Initialize Ollama LLM via LangChain
model = OllamaLLM(model=MODEL_NAME)

# **ZIP FILE EXTRACTION**

In [44]:
if not os.path.isdir(EXTRACT_DIR):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
    print(f"Extracted archive to '{EXTRACT_DIR}'")
else:
    print(f"Extraction directory '{EXTRACT_DIR}' already exists")

Extraction directory 'I-Soon-data' already exists


# **DATA TYPE CATEGORIZATION**

In [45]:
import os
import shutil
import re

# regex to detect any HTML tag
html_re = re.compile(r'<[A-Za-z/][^>]*>')

for root, _, files in os.walk(EXTRACT_DIR):
    for fname in files:
        ext = os.path.splitext(fname)[1].lower().lstrip('.') or 'no_extension'
        src_path = os.path.join(root, fname)

        if ext == 'md':
            # classify Markdown by content into md/html or md/non-html
            with open(src_path, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
            subfolder = 'html' if html_re.search(text) else 'non-html'
            dest_folder = os.path.join(ORGANIZED_DIR, 'md', subfolder)
        else:
            # preserve original extension grouping for non-MD
            dest_folder = os.path.join(ORGANIZED_DIR, ext)

        os.makedirs(dest_folder, exist_ok=True)
        dst_path = os.path.join(dest_folder, fname)
        shutil.copy2(src_path, dst_path)

print(f"Organized files under '{ORGANIZED_DIR}':\n"
      " • Markdown with HTML → md/html/\n"
      " • Markdown without HTML → md/non-html/\n"
      " • Other extensions → <extension>/")

Organized files under 'organized_data':
 • Markdown with HTML → md/html/
 • Markdown without HTML → md/non-html/
 • Other extensions → <extension>/


# **MD DATA TRANSFORMATIO TO JSON**

In [46]:
import os
import json
import shutil
from docling.document_converter import DocumentConverter

# Initialize converter
converter = DocumentConverter()

# Define input/output directories
input_dir = "organized_data/md/html"
output_dir = "html_markdown_to_json"
failed_dir = "failed_parsing"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(failed_dir, exist_ok=True)

# Process each Markdown file
for fname in os.listdir(input_dir):
    if fname.endswith(".md"):
        input_path = os.path.join(input_dir, fname)
        
        try:
            result = converter.convert(input_path)
            doc_dict = result.document.model_dump()
            
            # Output filename
            base = os.path.splitext(fname)[0]
            output_path = os.path.join(output_dir, f"{base}_docling.json")
            
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(doc_dict, f, indent=2)
            
            print(f"✅ Converted: {fname} → {output_path}")
        
        except Exception as e:
            # Move to failed folder
            shutil.move(input_path, os.path.join(failed_dir, fname))
            print(f"❌ Failed to convert {fname}: {e} (moved to {failed_dir})")

✅ Converted: dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md → html_markdown_to_json/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac_docling.json
✅ Converted: 28.md → html_markdown_to_json/28_docling.json
✅ Converted: 5.md → html_markdown_to_json/5_docling.json
✅ Converted: 38.md → html_markdown_to_json/38_docling.json
✅ Converted: 9d7bc879-3250-4013-ac04-5ff9bd6dff40.md → html_markdown_to_json/9d7bc879-3250-4013-ac04-5ff9bd6dff40_docling.json
✅ Converted: 18.md → html_markdown_to_json/18_docling.json
❌ Failed to convert ._3.md: 'utf-8' codec can't decode byte 0xfb in position 37: invalid start byte (moved to failed_parsing)
✅ Converted: 9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md → html_markdown_to_json/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b_docling.json
✅ Converted: 3348953d-66e9-4cac-8675-65bb5f2ef929.md → html_markdown_to_json/3348953d-66e9-4cac-8675-65bb5f2ef929_docling.json
✅ Converted: 1.md → html_markdown_to_json/1_docling.json
✅ Converted: 19.md → html_markdown_to_json/19_docling.json
✅ Converted: 07f

# **JSON TO DATAFRAME WITH AI**
# *Here we use AI to analyze all of the JSON files and identify which headers are usefull to keep. We store all the identified headers in a dictionary with the name of the json file as key and value the list contaning the headers*

In [50]:
import os
import json
import ast  # To safely evaluate header string to list

# Folder path
folder_path = "html_markdown_to_json"

# Store headers proposed for each file
file_headers = {}

# Loop through each file in the directory
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                doc_json = json.load(f)

            # Safely navigate the structure
            tables = doc_json.get("tables", [])
            if not tables:
                print(f"Skipped {filename}: no tables found.")
                continue

            cells = tables[0].get("data", {}).get("table_cells", [])
            sample_cells = cells[:20]  # Use a subset to limit input size

            # Format as JSON string
            cell_str = json.dumps(sample_cells, indent=2, ensure_ascii=False)

            # Prepare the prompt
            prompt = f"""
You are a law enforcement officer that specializes in investigating cybersecurity data leaks. Analyze the following JSON table cell data and infer what column headers would be appropriate
if this were converted into a pandas DataFrame. 

Only return a list of column names that are relevant to the data. Do not include any other text or explanation.

JSON data:
{cell_str}
"""

            # Get headers from the model
            headers_response = model(prompt)

            # Convert stringified list to actual Python list safely
            try:
                headers_list = ast.literal_eval(headers_response.strip())
                if isinstance(headers_list, list):
                    file_headers[filename] = headers_list
                else:
                    print(f"Warning: Invalid header format in {filename}")
            except Exception as eval_err:
                print(f"Header parsing error in {filename}: {eval_err}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Print the headers dictionary
for fname, headers in file_headers.items():
    print(f"{fname}: {headers}")

# Print the headers dictionary
print("\\n --- Headers Dicitonary --- \\n")
print(file_headers)

Skipped 9fe6b262-9944-417d-a0c4-9f2de1de2994_docling.json: no tables found.
Skipped 6d7fc7b3-c892-4cb5-bd4b-a5713c089d88_docling.json: no tables found.
Skipped 912204cb-8ab7-48b8-9abf-d803f3804d08_docling.json: no tables found.
Skipped b3031e66-40b6-45e8-9bcd-891dc1a280da_docling.json: no tables found.
Skipped dbc9c90e-a3e6-4d71-bb93-5fb8394095ac_docling.json: no tables found.
Skipped eda5b003-9250-4913-b724-74cca86240af_docling.json: no tables found.
Skipped 12756724-394c-4576-b373-7c53f1abbd94_docling.json: no tables found.
Skipped 5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_docling.json: no tables found.
Skipped 547aba02-6757-49c1-acb5-6df217cebfc7_docling.json: no tables found.
Skipped 54990932-71af-48dd-9a7a-2617b1407c54_docling.json: no tables found.
Skipped 585875ff-f8c5-4a02-acd7-fef37dc9ff11_docling.json: no tables found.
Skipped fe245192-1f9c-4f28-9b32-046fb7ce7e1e_docling.json: no tables found.
Skipped 3f451a52-d210-48d9-b56e-d28b9570bdc4_docling.json: no tables found.
Skipped 2db2

# **LLM ANALYSIS OF THE DICTIONARY OF HEADERS FROM JSON**
# *Here we use the LLM model's capability to analyze the dicitonary of headers and then use the most relevant headers*

In [ ]:
import json

# Convert the dictionary to a readable string
headers_str = json.dumps(file_headers, indent=2, ensure_ascii=False)

# Prompt the LLM to synthesize the best header list
prompt = f"""
You are a forensic data analyst reviewing metadata extracted from structured JSON communication logs.

The dictionary below maps filenames to lists of inferred DataFrame column headers. Your task is to review all these header lists and decide on a **single, unified list** of column names that would best represent the core information relevant to a cybersecurity investigation.

The list should be concise, semantically meaningful, and avoid redundant or purely structural fields unless they are essential for analysis.

Return only the final list in valid Python list syntax (e.g., ["Time", "From", "To", "Message"]) with no explanation.

Header dictionary:
{headers_str}
"""

# Send prompt to model
final_headers = model(prompt)

# Print the result
print("Final consolidated DataFrame headers:")
print(final_headers.strip())

Final consolidated DataFrame headers:
["Time", "From", "To", "Message"]


# **HERE WE CREATE A DATAFRAME FOR EVERY JSON FILE IN THE FOLDER HTML_MARKDOWN_TO_JSON**

In [53]:
import os
import json
import pandas as pd

# Folder with JSON files
folder_path = "html_markdown_to_json"

# Final headers from LLM
headers_response = ["Time", "From", "To", "Message"]

# Track DataFrame names
loaded_dataframe_names = []

# Loop through all JSON files
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        json_path = os.path.join(folder_path, filename)
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                doc = json.load(f)

            tables = doc.get("tables", [])
            if not tables:
                continue

            for i, table in enumerate(tables):
                cells = table.get("data", {}).get("table_cells", [])

                # Organize by row based on row indices
                rows_dict = {}
                for cell in cells:
                    row_idx = cell.get("start_row_offset_idx")
                    col_idx = cell.get("start_col_offset_idx")
                    text = cell.get("text", "")
                    if row_idx is not None and col_idx is not None:
                        if row_idx not in rows_dict:
                            rows_dict[row_idx] = {}
                        rows_dict[row_idx][col_idx] = text

                # Convert row dict to sorted list of rows
                sorted_rows = []
                for row_idx in sorted(rows_dict.keys()):
                    row = rows_dict[row_idx]
                    max_col = max(row.keys()) if row else -1
                    row_list = [row.get(col, "") for col in range(max_col + 1)]
                    sorted_rows.append(row_list)

                if not sorted_rows:
                    continue

                # Truncate headers to match the number of columns in the data
                header_cols = headers_response[:len(sorted_rows[0])]
                df = pd.DataFrame(sorted_rows, columns=header_cols)

                df_name = f"{filename} - table {i}"
                loaded_dataframe_names.append(df_name)

                print(f"\n=== {df_name} (shape: {df.shape}) ===")
                display(df)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Print names of all successfully loaded DataFrames
print("\n=== Loaded DataFrame Names ===")
for name in loaded_dataframe_names:
    print(name)


=== 28_docling.json - table 0 (shape: (47, 4)) ===


,Time,From,To,Message
0,Time,From,To,Message
1,2022-08-11 05:51:47,wxid_xusilpfkh31g21,Shutd0wn,吴总，现在方便吗，想和你通个电话，有个事情汇报一下
2,2022-08-11 05:51:59,Shutd0wn,wxid_xusilpfkh31g21,嗯
3,2022-08-11 05:52:40,Shutd0wn,wxid_xusilpfkh31g21,？
4,2022-08-11 06:29:08,wxid_xusilpfkh31g21,Shutd0wn,花雨夜和! 4p47hy的聊天记录
5,2022-08-11 06:33:23,Shutd0wn,wxid_xusilpfkh31g21,嗯，那他也不知道
6,2022-08-11 06:38:04,wxid_xusilpfkh31g21,Shutd0wn,是的
7,2022-08-11 07:46:23,Shutd0wn,wxid_xusilpfkh31g21,她说了哪些信息点让你觉得知道的太多了，我评估评估
8,2022-08-11 07:51:14,wxid_xusilpfkh31g21,Shutd0wn,1、规定这样的报销制度的原因到底是如何：如是公司资金紧张而不发，那为什么还要组织团建，而且团...
9,2022-08-11 07:52:08,wxid_xusilpfkh31g21,Shutd0wn,2、奇安信之前说要入股，因为20年没法年终奖导致人心涣散，最终很多核心人员被奇安信挖走，然后...



=== 19_docling.json - table 0 (shape: (22, 4)) ===


,Time,From,To,Message
0,Time,From,To,Message
1,2021-11-30 02:58:46,ibabaimama,wxid_12n748um1thl21,客户看了，对菲关注度不大，能搞越南或缅甸不？
2,2021-11-30 02:59:51,wxid_12n748um1thl21,ibabaimama,好的我问问
3,2021-11-30 06:02:07,wxid_12n748um1thl21,ibabaimama,我目前只掌握啦越南社科院的
4,2021-11-30 06:02:23,wxid_12n748um1thl21,ibabaimama,[破涕为笑]鸡肋部门
5,2021-11-30 06:21:07,ibabaimama,wxid_12n748um1thl21,[捂脸]看看电信的能搞不，vitta
6,2021-11-30 06:22:03,wxid_12n748um1thl21,ibabaimama,嗯，之前进去过
7,2021-11-30 06:22:12,wxid_12n748um1thl21,ibabaimama,第二天就被扫出来了
8,2021-11-30 06:22:21,wxid_12n748um1thl21,ibabaimama,我好熟悉这个名字
9,2021-11-30 06:22:25,wxid_12n748um1thl21,ibabaimama,[呲牙]



=== 29_docling.json - table 0 (shape: (81, 4)) ===


...


=== 9_docling.json - table 0 (shape: (236, 4)) ===


,Time,From,To,Message
0,Time,From,To,Message
1,2021-09-09 05:17:43,wxid_7p054rmzkhqf21,gzp1991101,你回来了么
2,2021-09-09 05:24:13,gzp1991101,wxid_7p054rmzkhqf21,还没
3,2021-09-09 05:24:20,wxid_7p054rmzkhqf21,gzp1991101,明天么
4,2021-09-09 05:24:27,gzp1991101,wxid_7p054rmzkhqf21,这样打算的
...,...,...,...,...
231,2021-09-13 15:59:47,wxid_7p054rmzkhqf21,gzp1991101,a感觉用不到，主要是其他行业，看看找点机会
232,2021-09-13 16:00:04,gzp1991101,wxid_7p054rmzkhqf21,啥时候跳槽
233,2021-09-13 16:00:20,wxid_7p054rmzkhqf21,gzp1991101,快了，我今天提了，估计最近
234,2021-09-13 16:00:59,gzp1991101,wxid_7p054rmzkhqf21,太难搞了



=== 38_docling.json - table 0 (shape: (22, 4)) ===


,Time,From,To,Message
0,Time,From,To,Message
1,2022-05-17 06:12:24,wxid_zbytkn4qjl3r22,lengmo,渗透测试三组——魁杓攻防研究实验室，\n 魁杓，指北斗七星，出自《淮南子·天文训...
2,2022-05-17 06:12:39,wxid_zbytkn4qjl3r22,lengmo,三组有内涵，名字可以哈？
3,2022-05-17 06:13:36,lengmo,wxid_zbytkn4qjl3r22,嗯
4,2022-05-17 06:13:43,lengmo,wxid_zbytkn4qjl3r22,要换名字？
5,2022-05-17 06:14:16,wxid_zbytkn4qjl3r22,lengmo,二组和三组，后面都以实验室运作，现在也没了1组，到B栋来了都要换个名字
6,2022-05-17 06:15:18,lengmo,wxid_zbytkn4qjl3r22,哦
7,2022-05-23 03:37:37,lengmo,wxid_zbytkn4qjl3r22,那个售前的事情，我和周伟伟聊了下··
8,2022-05-23 03:38:48,lengmo,wxid_zbytkn4qjl3r22,觉得他接也行，刚好生态管理部和售前也是一条线的，售前这块接触的厂商比较多，也可以反馈。
9,2022-05-23 04:11:27,wxid_zbytkn4qjl3r22,lengmo,和吴总沟通了下，他觉得是有必要，但还是比较顾虑整个团队的磨合和整体稳定性，方案1我们可以同步...



=== Loaded DataFrame Names ===
28_docling.json - table 0
19_docling.json - table 0
29_docling.json - table 0
18_docling.json - table 0
15_docling.json - table 0
40_docling.json - table 0
12_docling.json - table 0
24_docling.json - table 0
23_docling.json - table 0
6_docling.json - table 0
1_docling.json - table 0
36_docling.json - table 0
31_docling.json - table 0
13_docling.json - table 0
41_docling.json - table 0
14_docling.json - table 0
7_docling.json - table 0
30_docling.json - table 0
37_docling.json - table 0
22_docling.json - table 0
16_docling.json - table 0
11_docling.json - table 0
35_docling.json - table 0
32_docling.json - table 0
5_docling.json - table 0
2_docling.json - table 0
27_docling.json - table 0
20_docling.json - table 0
10_docling.json - table 0
17_docling.json - table 0
21_docling.json - table 0
26_docling.json - table 0
33_docling.json - table 0
34_docling.json - table 0
3_docling.json - table 0
4_docling.json - table 0
39_docling.json - table 0
9_docling.jso

# **DATAFRAMEs TO 1 CSV FOR CHAT ANALYSIS**

In [54]:
import os
import json
import pandas as pd

# Folder with JSON files
folder_path = "html_markdown_to_json"

# Final headers from LLM
headers_response = ["Time", "From", "To", "Message"]

# Track all DataFrames for merging
all_dfs = []

# Loop through all JSON files
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        json_path = os.path.join(folder_path, filename)
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                doc = json.load(f)

            tables = doc.get("tables", [])
            if not tables:
                continue

            for i, table in enumerate(tables):
                cells = table.get("data", {}).get("table_cells", [])

                # Organize cells by row index
                rows_dict = {}
                for cell in cells:
                    row_idx = cell.get("start_row_offset_idx")
                    col_idx = cell.get("start_col_offset_idx")
                    text = cell.get("text", "")
                    if row_idx is not None and col_idx is not None:
                        if row_idx not in rows_dict:
                            rows_dict[row_idx] = {}
                        rows_dict[row_idx][col_idx] = text

                # Sort rows and fill missing columns
                sorted_rows = []
                for row_idx in sorted(rows_dict.keys()):
                    row = rows_dict[row_idx]
                    max_col = max(row.keys()) if row else -1
                    row_list = [row.get(col, "") for col in range(max_col + 1)]
                    sorted_rows.append(row_list)

                if not sorted_rows:
                    continue

                # Align header with row width
                header_cols = headers_response[:len(sorted_rows[0])]
                df = pd.DataFrame(sorted_rows, columns=header_cols)

                # Optionally tag source
                df["SourceFile"] = f"{filename} - table {i}"

                # Store the DataFrame
                all_dfs.append(df)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Merge all into a single DataFrame
if all_dfs:
    merged_df = pd.concat(all_dfs, ignore_index=True)
    merged_df.to_csv("combined_output.csv", index=False)
    print("✅ All data saved to 'combined_output.csv'")
else:
    print("⚠️ No DataFrames were loaded.")

✅ All data saved to 'combined_output.csv'
